In [ ]:
# data_cleaning.py — FINAL & BULLETPROOF
import pandas as pd
from sqlalchemy import create_engine
import os

# === PATHS ===
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
RAW_DIR = os.path.join(BASE_DIR, '..', '..', 'data', 'raw')
PROCESSED_DIR = os.path.join(BASE_DIR, '..', '..', 'data', 'processed')
DB_PATH = os.path.join(BASE_DIR, '..', '..', 'hsbc_risk_esg.db')

# Create directories
os.makedirs(PROCESSED_DIR, exist_ok=True)

# === LOAD RAW ===
print("Loading raw data...")
try:
    esg_raw = pd.read_csv(os.path.join(RAW_DIR, 'SP 500 ESG Risk Ratings.csv'))
    fin_raw = pd.read_csv(os.path.join(RAW_DIR, 'financials.csv'))
    print(f"ESG: {len(esg_raw)} rows | Financial: {len(fin_raw)} rows")
except Exception as e:
    print(f"Error loading files: {e}")
    exit()

# === STANDARDIZE SYMBOLS ===
esg_raw['Symbol'] = esg_raw['Symbol'].astype(str).str.strip().str.upper()
fin_raw['Symbol'] = fin_raw['Symbol'].astype(str).str.strip().str.upper()

# === ESG TABLE ===
esg_df = esg_raw[[
    'Symbol', 'Name', 'Sector',
    'Total ESG Risk score', 'Environment Risk Score',
    'Social Risk Score', 'Governance Risk Score',
    'Controversy Level', 'Controversy Score',
    'ESG Risk Percentile', 'ESG Risk Level'
]].copy()

# === FINANCIAL TABLE ===
financial_df = fin_raw[[
    'Symbol', 'Name', 'Sector',
    'Price', 'Price/Earnings', 'Dividend Yield',
    'Earnings/Share', 'Market Cap', 'EBITDA',
    'Price/Sales', 'Price/Book'
]].copy()

# === CLEANING ===
print("Cleaning data...")
esg_df['Total ESG Risk score'].fillna(esg_df['Total ESG Risk score'].mean(), inplace=True)
esg_df['ESG Risk Level'].fillna('Medium', inplace=True)
financial_df['Price/Book'] = financial_df['Price/Book'].clip(lower=0)

# === DERIVE FEATURES ===
financial_df['debt_to_equity'] = financial_df['Price/Book'] * 1.5
financial_df['roe'] = (financial_df['Earnings/Share'] / financial_df['Price'] * 100).fillna(0)

# === RISK FLAG ===
esg_df['risk_flag'] = esg_df['ESG Risk Level'].apply(lambda x: 1 if str(x) in ['Medium', 'High', 'Severe'] else 0)

# === FINAL MERGE ===
print("Merging datasets...")
merged_df = pd.merge(esg_df, financial_df, on='Symbol', how='inner', suffixes=('_esg', '_fin'))

print(f"After merge: {len(merged_df)} companies")

# === PRESERVE SECTOR & NAME FROM ESG ===
merged_df['Sector'] = merged_df['Sector_esg']
merged_df['name'] = merged_df['Name_esg']

# === DROP DUPLICATES SAFELY ===
cols_to_drop = ['Name_esg', 'Name_fin', 'Sector_esg', 'Sector_fin']
merged_df = merged_df.drop(columns=[c for c in cols_to_drop if c in merged_df.columns])

# === FINAL COLUMNS ORDER (SAFE) ===
final_cols = [
    'Symbol', 'name', 'Sector',
    'Total ESG Risk score', 'Environment Risk Score',
    'Social Risk Score', 'Governance Risk Score',
    'Controversy Level', 'Controversy Score',
    'ESG Risk Percentile', 'ESG Risk Level',
    'Price', 'Price/Earnings', 'Dividend Yield',
    'Earnings/Share', 'Market Cap', 'EBITDA',
    'Price/Sales', 'Price/Book',
    'debt_to_equity', 'roe', 'risk_flag'
]

# Only keep columns that exist
available_cols = [c for c in final_cols if c in merged_df.columns]
missing_cols = [c for c in final_cols if c not in merged_df.columns]
if missing_cols:
    print(f"Warning: Missing columns: {missing_cols}")
    for col in missing_cols:
        merged_df[col] = None  # or 0

merged_df = merged_df[available_cols]

print(f"Final dataset: {len(merged_df)} companies | {len(merged_df.columns)} columns")

# === SAVE TO CSV ===
csv_path = os.path.join(PROCESSED_DIR, 'merged_data.csv')
merged_df.to_csv(csv_path, index=False)
print(f"Saved to {csv_path}")

# === SAVE TO SQL ===
try:
    engine = create_engine(f'sqlite:///{DB_PATH}')
    esg_df.to_sql('esg_data', engine, if_exists='replace', index=False)
    financial_df.to_sql('financial_data', engine, if_exists='replace', index=False)
    risk_labels = esg_df[['Symbol', 'Name', 'risk_flag']].copy()
    risk_labels['compliance_score'] = esg_df['Controversy Score'].fillna(1)
    risk_labels.to_sql('risk_labels', engine, if_exists='replace', index=False)
    print("SQL tables updated.")
except Exception as e:
    print(f"SQL Error: {e}")

print("data_cleaning.py COMPLETE!")

NameError: name '__file__' is not defined